### Compute Total Water Storage from PFCLM outputs

**PFTools Python**:  
`TWS = ( calculate_subsurface_storage + calculate_surface_storage + SWE (STORAGE?) ) / 24`   
AKA (with daily averages):  
`TWS = SUBstorage + SURFstorage + SWEstorage`

_EL: SWE storage would be swe from clm output * 1000 (mm->m)dxdy_
_____

This script takes hourly PFCLM outputs as PFB files and computes the daily, (monthly, and yearly) averages to be saved as PFB files.

Inputs:
- Directory where PFCLM daily averages are and directory where you want to save output (these should be the same)
- Hourly PFB files of subsurface storage, surface water storage and CLM Snow Water Equivalent outputs
- water year and day start/end???

Outputs:
- PFB files for daily average Total Water Storage

    
Notes (10/26/22):
- DO WE NEED TO multiply SWE (or anything else) by dy * dx??
- Because the daily files are read in here and have already been corrected for the CONUS timezone, no adjustments here are necessary
- Need to add in MONTHLY averages


In [1]:
import numpy as np
from parflow import Run
import sys
from parflow.tools.io import read_pfb,write_pfb
import parflow.tools.hydrology as hydro

In [16]:
#these 3 entries (year, day start and day end) will eventually be argv to the script so that it can be run from bash script
water_year = 1999
day_start = 0 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 365 #day_end = 365 is the final day of the water year, Sept 30

# water_year = int(sys.argv[1])
# day_start = int(sys.argv[1])
# day_end = int(sys.argv[1])

# PFCLM run name
#runname = f'Taylor_{water_year}' #f'CONUS2_{water_year}'

# directory to save averages to
#directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
directory_out = '/home/dtt2/CONUS2/analysis_scripts/Taylor_test_outputs'

####### TAYLOR ########

nz = 5 #10
ny = 47 #3256
nx = 45 #4442

dx = 1000
dy = 1000
dz = 5 # 200

# apparently it's good to use high numbers when saving files to speed up reading?
# for write_pfb function
p = 5 #72
q = 5 #48
r = 1


In [26]:
tws = np.zeros((ny,nx))
tws_m = np.zeros((ny,nx))
tws_y = np.zeros((ny,nx))

for day in range(day_start+1,day_end+1): 
    
    timestamp_day = str(int(day)).rjust(3, '0')
    print(f'Reading storage from day {timestamp_day}')
    
    
    # Read in Subsurface Storage PFBs (5 layers)
    sub_storage_pfb = read_pfb(f'{directory_out}/SUBstorage.{water_year}.daily.{timestamp_day}.pfb')
    # sum all layers of subsurface storage
    sub_storage_sum = np.sum(sub_storage_pfb, axis = 0)

    
    # Read in Surface Storage PFBs
    surf_storage_pfb = np.squeeze(read_pfb(f'{directory_out}/SURF_WATstorage.{water_year}.daily.{timestamp_day}.pfb'))

    
    # Read in SWE PFBs
    swe_pfb =  np.squeeze(read_pfb(f'{directory_out}/swe_out.{water_year}.daily.{timestamp_day}.pfb'))
    # SWE is in mm, so convert to m
    swe_meters = swe_pfb*1000

    
    # Calculate Total Water Storage
    tws = sub_storage_sum + surf_storage_pfb + swe_pfb
    print(tws.shape)
    # save daily TWS PFB
    write_pfb(f'{directory_out}/TWS.{water_year}.daily.{timestamp_day}.pfb',tws,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    
    
    # monthly TWS
    
    
    # yearly TWS
    tws_y += tws / 365

# save yearly TWS PFB
write_pfb(f'{directory_out}/TWS.{water_year}.yearly.pfb',tws_y,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)


Reading storage from day 001
(47, 45)
Reading storage from day 002
(47, 45)
Reading storage from day 003
(47, 45)
Reading storage from day 004
(47, 45)
Reading storage from day 005
(47, 45)
Reading storage from day 006
(47, 45)
Reading storage from day 007
(47, 45)
Reading storage from day 008
(47, 45)
Reading storage from day 009
(47, 45)
Reading storage from day 010
(47, 45)
Reading storage from day 011
(47, 45)
Reading storage from day 012
(47, 45)
Reading storage from day 013
(47, 45)
Reading storage from day 014
(47, 45)
Reading storage from day 015
(47, 45)
Reading storage from day 016
(47, 45)
Reading storage from day 017
(47, 45)
Reading storage from day 018
(47, 45)
Reading storage from day 019
(47, 45)
Reading storage from day 020
(47, 45)
Reading storage from day 021
(47, 45)
Reading storage from day 022
(47, 45)
Reading storage from day 023
(47, 45)
Reading storage from day 024
(47, 45)
Reading storage from day 025
(47, 45)
Reading storage from day 026
(47, 45)
Reading stor

In [22]:
tws[30,30]

34289857.551114544

In [23]:
tws_y[30,30]

34293190.83671028

In [ ]:
# Read in Subsurface Storage pfb's (5 layers)
sub_storage_pfb = read_pfb(f'{directory_out}/SUBstorage.{water_year}.daily.{timestamp_day_out}.pfb')
print(sub_storage_pfb.shape)

# sum all layers of subsurface storage
sub_storage_sum = np.sum(sub_storage_pfb, axis = 0)
print(sub_storage_sum.shape)

In [ ]:
# Read in Surface Storage pfb's
surf_storage_pfb = np.squeeze(read_pfb(f'{directory_out}/SURF_WATstorage.{water_year}.daily.{timestamp_day_out}.pfb'))
print(surf_storage_pfb.shape)

In [ ]:
# Read in SWE pfb's
swe_pfb =  np.squeeze(read_pfb(f'{directory_out}/swe_out.{water_year}.daily.{timestamp_day_out}.pfb'))
print(swe_pfb.shape)

# SWE is in mm, so convert to m
swe_meters = swe_pfb*1000

In [ ]:
tws = sub_storage_sum + surf_storage_pfb + swe_pfb
print(tws.shape)

In [ ]:
write_pfb(f'{directory_out}/TWS.{water_year}.daily.{timestamp_day_out}.pfb',tws,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)

In [ ]:
print(tws[30,30])

In [ ]:
sub_storage_sum[30,30] + surf_storage_pfb[30,30] + swe_pfb[30,30]

In [ ]:
#NCLMOUTPUTS = 13 + 4 #13 (number variables) + number of layers over which CLM is active, NZ root

#these 3 entries (year, day start and day end) will eventually be argv to the script so that it can be run from bash script
water_year = 2003
day_start = 0 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 3 #day_end = 364 is the final day of the water year, Sept 30

# water_year = int(sys.argv[1])
# day_start = int(sys.argv[1])
# day_end = int(sys.argv[1])

# path to PF outputs outputs
path_outputs = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/' #f'/WY{water_year}/'

# PFCLM run name
runname = 'spinup.wy2003' #f'CONUS2_{water_year}'

# directory to save averages to
directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'

In [ ]:
run = Run.from_definition(f'{path_outputs}/{runname}.pfidb')
data = run.data_accessor

porosity = data.computed_porosity 
specific_storage = data.specific_storage 
mannings = data.mannings

## remove input filenames for TopoSlopes to force the data accessor to read the output slopes
## this fixes a windows issue
run.TopoSlopesX.FileName = None
run.TopoSlopesY.FileName = None

slopex = data.slope_x 
slopey = data.slope_y 
mask = data.mask

# formatting the mask so that values outside the domain are NA and inside the domain are 1
# check with mask that has 0 and 1
active_mask=mask.copy()
active_mask[active_mask > 0] = 1

In [ ]:
###READING ALL STATIC VARIABLES NEEDED
# Read in porosity data
#porosity = read_pfb(f'{path_outputs}{runname}.out.porosity.pfb')
#...
#etc.

#nz,ny,nx = porosity.shape()

nz = 10
ny = 3256
nx = 4442

dx = 1000
dy = 1000
dz = 200
dz_3d = data.dz

# apparently it's good to use high numbers when saving files to speed up reading?
# for write_pfb function
p = 72
q = 48
r = 1

data.time
# #list of clm variables you want
# variables_clm = ['eflx_lh_tot','qflx_evap_grnd','qflx_tran_veg','swe_out','t_grnd','t_soil']
# #indication whether you want the mean (1) or the sum (0)
# variables_clm_mean = [0,0,0,1,1,1]

# ALL_CLM = ['eflx_lh_tot','eflx_lwrad_out','eflx_sh_tot','eflx_soil_grnd','qflx_evap_tot','qflx_evap_grnd','qflx_evap_soi','qflx_evap_veg','qflx_tran_veg','qflx_infl','swe_out','t_grnd','qflx_qirr','t_soil']

In [ ]:
porosity.shape

In [ ]:
active_mask.shape

In [ ]:
for day in range(day_start,day_end):

    timestamp_day_out = str(int(day+1)).rjust(3, '0')

    ##INITIALIZE WHATEVER DYNAMIC VARIABLES THAT NEED HOURLY READING
    overland_flow = np.zeros((ny, nx)) 
    soil_moisture = np.zeros((nz,ny,nx))
    wtd = np.zeros((ny, nx)) 
    
    # Subsurface Storage Components
    subsurface_storage = np.zeros((nz,ny,nx)) # Total Subsurface Storage
    
    # Total Water Storage Components
    surface_storage = np.zeros((ny,nx)) 
    
    
    #if not variables_clm == False:
    #    clm_output = np.zeros((NCLMOUTPUTS,ny,nx))
    for h in range(day*24+1,(day+1)*24+1):
        timestamp_reading = str(int(h)).rjust(5, '0')
        
        #read pressure and saturation at timestep 
        saturation = read_pfb(f'{path_outputs}{runname}.out.satur.{timestamp_reading}.pfb') * active_mask
#        pressure = read_pfb(f'{path_outputs}{runname}.out.press.{timestamp_reading}.pfb') * active_mask
        print(f'reading {path_outputs}{runname}.out at time {timestamp_reading}')
        
        ################### 
        # Computations
        ###################
        
        #Soil Moisture
        soil_moisture += saturation * porosity
        
        # Subsurface Storage
#        subsurface_storage += hydro.calculate_subsurface_storage(porosity, pressure, saturation, specific_storage, dx, dy, dz_3d, mask = active_mask)
        
        # Surface Storage
        ## total surface storage for this time step is the summation of substorage surface across all x/y slices <-- from other script, is this still TRUE??
#        surface_storage += hydro.calculate_surface_storage(pressure, dx, dy, mask = active_mask)
        
        # Water Table Depth
#        wtd = hydro.calculate_water_table_depth(pressure, saturation, dz_3d)
        
        # Flow [m^3/h]
#        overland_flow = hydro.calculate_overland_flow_grid(pressure, slopex, slopey, mannings, dx, dy, mask = active_mask)
        
        

        #CLM Variables
        #clm_output += read_pfb(f'{path_outputs}{runname}.out.clm_output.{timestamp_reading}.C.pfb')

    ### compute average for average variables
    soil_moisture /= 24
    subsurface_storage /= 24
    surface_storage /= 24
    wtd /= 24 # CHANGE THIS TO BE ACCUMULATED?? 10/7/22
    overland_flow /= 24


    subsurface[active_mask==0]=-10**(38)
    ### SAVE VARIABLES AS PFB FILES
    write_pfb(f'{directory_out}/SM.{water_year}.daily.{timestamp_day_out}.pfb',soil_moisture,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    # IS THIS 'GWstore' OR IS IT 'storage'
    write_pfb(f'{directory_out}/GWstorage.{water_year}.daily.{timestamp_day_out}.pfb',subsurface_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    write_pfb(f'{directory_out}/surf_wat.{water_year}.daily.{timestamp_day_out}.pfb',surface_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    write_pfb(f'{directory_out}/WTd.{water_year}.daily.{timestamp_day_out}.pfb',wtd,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    write_pfb(f'{directory_out}/flow.{water_year}.daily.{timestamp_day_out}.pfb',overland_flow,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
